In [11]:
import sys
sys.path.append('./utils/')
import pandas as pd
import numpy as np
import os 
import cv2
from skimage.io import imread,imshow
from utils_fcn import DataGeneratorFCN
import matplotlib.pyplot as plt
import keras.callbacks
%matplotlib inline

In [12]:
dataset = pd.read_csv(os.path.join('/home','wvillegas','dataset-mask', 'full_masks.csv'))

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X_train, X_test, Y_train, Y_test = train_test_split(dataset['orig'],dataset['mask'],
                                                    test_size = 0.2, random_state=1)

In [15]:
partition = {'train':list(X_train),
             'test': list(X_test)}

In [16]:
img_list = list(X_train) + list(X_test)
mask_list = list(Y_train) + list(Y_test)

In [17]:
labels = dict(zip(img_list, mask_list))

In [18]:
masks_path = os.path.join('/home','wvillegas','dataset-mask','dataset_resize','masks_resize')
img_path = os.path.join('/home','wvillegas','dataset-mask','dataset_resize','images_resize')

In [19]:
batch_size = 1
# dim_img = (1728, 2304)
# dim_mask = (658,874)

In [20]:
train_generator = DataGeneratorFCN(batch_size=batch_size,img_path=img_path,
                                labels=labels,list_IDs=partition['train'],n_channels=3,
                                n_channels_label=1,shuffle=True,mask_path=masks_path)
valid_generator = DataGeneratorFCN(batch_size=batch_size,img_path=img_path,
                                labels=labels,list_IDs=partition['test'],n_channels=3,
                                n_channels_label=1,shuffle=True,mask_path=masks_path)

In [21]:
from keras.applications import VGG16
from keras.layers import Conv2DTranspose,Conv2D, Add, Cropping2D
from keras import Model
from keras.models import load_model

In [22]:
net = VGG16(include_top=False, weights='imagenet')
# net.load_weights('/home/wvillegas/DLProjects/BudClassifier/cmdscripts/modelosV2/mobilenet_weights_detection.h5', by_name=True)

In [23]:
net.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [24]:
# test arquitectura paper de FCN 
# deconv1 = Conv2D(filters=256,kernel_size=3,strides=1,activation='relu')(mobilenet.output)
predict = Conv2D(filters=1,kernel_size=1,strides=1)(net.output)
deconv1 = Conv2DTranspose(filters=512,kernel_size=4,strides=2, padding='same')(predict)
predict_pool4 = Conv2D(filters=1,kernel_size=1,strides=1)(net.get_layer('block4_pool').output)
add1 = Add()([deconv1, predict_pool4])
deconv2 = Conv2DTranspose(filters=1,kernel_size=32,strides=16,padding='same')(add1)

In [25]:
fcn = Model(inputs=net.input,outputs=deconv2)

In [26]:
# fcn.summary()

In [27]:
# for layer in fcn.layers:
#     layer.trainable = False

In [28]:
# for layer in fcn.layers:
#     print(layer.trainable)

In [29]:
# plt.imshow(pred[0,:,:,0], cmap="gray")

In [30]:
# Xtrain = np.asarray(Xtrain)
# Ytrain = np.asarray(Ytrain)
# Xtest = np.asarray(Xtest)
# Ytest = np.asarray(Ytest)

In [31]:
from keras.callbacks import Callback

In [32]:
class LossAccHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.accuracy = []
        self.val_acc = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.accuracy.append(logs.get['acc'])
        self.val_acc.append(logs.get['val_acc'])
        

In [33]:
# callback = LossAccHistory()

In [34]:
from keras.optimizers import SGD, RMSprop

In [35]:
sgd = SGD(lr=0.01,momentum=0.9)
fcn.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [36]:
# reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
#                               patience=3, min_lr=0.00001)

In [ ]:
history = fcn.fit_generator(generator=train_generator,validation_data=valid_generator,
                            use_multiprocessing=True,workers=6, epochs=1)

Epoch 1/1


In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
list_img = partition['test'][93:94]

In [ ]:
inp_img = []
for img in list_img:
    temp = cv2.imread(img_path + '/' + img)
    temp = cv2.resize(temp, (0,0), fx=0.5, fy=0.5)
#   temp = cv2.normalize(temp, temp, 0, 255, cv2.NORM_MINMAX)
    inp_img.append(temp)
    
ground_truth = []
for img in list_img:
    mask = labels[img]
    print(mask)
    temp = cv2.imread(masks_path + '/' + mask)
    temp = cv2.cvtColor(temp, cv2.COLOR_BGR2GRAY)
    temp = temp.astype(bool).astype(int)
    ground_truth.append(temp)


In [ ]:
for i in np.arange(len(inp_img)):
    print(inp_img[i].shape)

In [ ]:
inp_img = np.asarray(inp_img)
ground_truth = np.asarray(ground_truth)

In [ ]:
inp_img.shape

In [ ]:
pred = fcn.predict(inp_img)

In [ ]:
# img = cv2.imread(img_path + '/0717.jpg')
# img = cv2.resize(img, (0,0), fx=0.5, fy=0.5)
# img.shape

In [ ]:
# xx = np.empty((1,img.shape[0],img.shape[1], 3))
# xx[0,] = img

In [ ]:
# pred = fcn.predict(xx)
# pred.shape

In [ ]:
pred.shape

In [ ]:
imshow(pred[0,:,:,0])

In [ ]:
ground_truth.shape

In [ ]:
imshow(ground_truth[0,:,:])

In [ ]:
ground_truth[1,:,:,0].sum()